## Import Modules

In [ ]:
import pdal
import sys
import os
import json
sys.path.append("../../../PythonScripts")
from glob import glob
import open3d as o3d
import os
import numpy as np
import pandas as pd
from multiprocessing import Pool
from pipeline_functions import downscale_las, get_scales, downscale_ply, getFeatures,getFeaturesParallel,Cylinder
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from time import time
# from scipy.spatial import cKDTree
from sklearn.neighbors import KDTree, BallTree
# from scipy.spatial import cKDTree
import pickle
from datetime import datetime, timezone
from itertools import repeat # Used for starmap function
import gc # Used to free up memory to prevent kernel restarting

sns.set()

# Get number of processors to run loop

In [ ]:
processors = os.cpu_count() - 8

## Get File to run

In [ ]:
# ROOT = """/home/sspiegel/CapstoneData/WashingtonDC/OpenDataDC_LAS_Point_Cloud_2020_Block1/Block1/2117.las"""
ROOT = """/home/sspiegel/CapstoneData/Paris/training_10_classes/Lille2.ply"""
# ROOT = """/home/sspiegel/CapstoneData/PA/225019325.las"""

## Load in base point cloud

In [ ]:
pipeline_json = [
    {
        "type": "readers.ply",
        "filename": ROOT
    }
        
]


pipeline = pdal.Pipeline(json.dumps(pipeline_json))

# Execute the pipeline
# This will process the data according to the stages defined in the pipeline
pipeline.execute()

xy = pipeline.arrays[0]

mta = pipeline.metadata



In [ ]:
xyzT = np.array((xy["X"], xy["Y"], xy["Z"])).T

In [ ]:
xyz = xyzT - np.min(xyzT, axis = 0)

## Get Classification

In [ ]:
cls = xy["Classification"].astype(int)

In [ ]:
scls = get_scales(S = 6) # Use base paprameters from function

In [ ]:
scls

## Instantiate point cloud

In [ ]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)

In [ ]:
pc_dict = []
for s in scls:
    
    pc = pcd.voxel_down_sample(s[1])
    # pc_array = np.asarray(pc.points)
    pc_dict.append({
      "r" : s[0],
     "grid_size" : s[1],
     "point_cloud" : pc
     # "point_cloud_array" : pc_carray
    })


## Get scaling factors

### Downscale point clouds: Methodology

* Downscale point clouds based on multiscaling factor
* The scaling factor chosen from [Thomas Hugues et. al](https://ieeexplore.ieee.org/document/8490990/)
* Parameters
  * initial radius ($r_0$): 0.1
  * Number of scales ($S$): 6
  * Base exponent for expanding radius ($\gamma$): 2
  * scaling factor of grid ($\rho$): 5
  * radius of sphere for scale $s$: $r_s = r_0 * \gamma^{s}$
  * downsampled voxel size: $\frac{r_s}{\rho}$

* Use multiprocessing to speed up process

### Sample point cloud (first radius)

In [ ]:
test = pc_dict[0]

test_pc = test["point_cloud"]

cyl = Cylinder(R = test['r'])

tree = KDTree(test_pc.points)
# tree = cKDTree(CylPoints)

In [ ]:
print(test)

In [ ]:
test_pc_arr = np.asarray(test_pc.points)

## Get Distance matrix

In [ ]:
# cylPCDPoints = np.asarray(pcd.points)[:,:2]

In [ ]:
distMat = tree.query_radius(pcd.points, r=test["r"])



# distMat = tree.query_ball_point(pcd.points, r=test["r"], workers = 1)


In [ ]:
# distMat

In [ ]:
# args1 = zip(repeat(pcOut),repeat(test_pc_arr), distMat)
# with Pool(processes=processors) as pool1:
#     result1 = pool1.starmap(cyl.computePoints, args1)
# distMatFix = np.array(result1)

In [ ]:
pcOut = np.asarray(pcd.points)


In [ ]:
del pc_dict, xyzT, xyz, xy, pcd

gc.collect()

In [ ]:
# cyl.computePointsParallel(pcOut[0],test_pc_arr,distMat[0])

In [ ]:
args1 = zip(pcOut,repeat(test_pc_arr), distMat)
with Pool(processes=processors) as pool1:
    result1 = pool1.starmap(cyl.computePointsParallel, args1)
distMatFix = np.asarray(result1, dtype = "object")

In [ ]:
gc.collect()

In [ ]:
# distMat.dtype

In [ ]:
# distMatFix = np.asarray(result1, dtype = "object")

In [ ]:
# distMatFix[0]
# # 

In [ ]:
# distMat[0]

In [ ]:
# pcOut[0]

In [ ]:
# ids = np.random.randint(0, pcOut.shape[0], 1)

In [ ]:
# ids

In [ ]:
# pcCenter = o3d.geometry.PointCloud()
# pcDown = o3d.geometry.PointCloud()
# pcCyl = o3d.geometry.PointCloud()

# pcCenter.points = o3d.utility.Vector3dVector(pcOut[ids[0]].reshape(1,-1))
# pcDown.points = o3d.utility.Vector3dVector(test_pc_arr[distMat[ids[0]]])
# pcCyl.points = o3d.utility.Vector3dVector(test_pc_arr[distMatFix[ids[0]]])

# pcCenter.paint_uniform_color([1,0,0])
# pcDown.paint_uniform_color([0,1,0])
# pcCyl.paint_uniform_color([0,0,1])

# o3d.visualization.draw_geometries([pcCenter, pcDown, pcCyl])


In [ ]:
del distMat

In [ ]:
gc.collect()

## Split and apply features algorithmKDTree

In [ ]:
# DIV = 100000
# groups = distMatFix.shape[0] // DIV

In [ ]:
s = time()

bigList = []

# for g in range(groups + 1):
#     print("""Processing batch %d...""" % (g+1))
#     practiceList = distMatFix[g*DIV:(g+1)*DIV]

args = zip(repeat(test_pc_arr), distMatFix)
with Pool(processes=processors) as pool:
    bigList = pool.starmap(getFeaturesParallel, args)
    # result = np.array(bigList)
    # bigList.append(result)
    # print(practiceList.shape)
    # break

bigList = np.array(bigList)
e = time()

print("""\n\n\n\nDone!!!  Total time: %.4f seconds""" % (e - s))

In [ ]:
bigList[bigList[:,-1] >= 5 ]

## Save as a pickle file

In [ ]:
del distMatFix, tree

gc.collect()

In [ ]:
# pickleOut = f"""/home/sspiegel/CapstoneData/Paris/Toronto_3D/pickleFiles/{datetime.now(timezone.utc).strftime("%Y_%m_%dT%H_%M")}_L001_r_{str(test["r"]).split('.')[0]}_{str(test["r"]).split('.')[1]}_grid_{str(test["grid_size"]).split('.')[0]}_{str(test["grid_size"]).split('.')[1]}.pkl"""

In [ ]:
getTime = datetime.now(timezone.utc).strftime("%Y_%m_%dT%H_%M")

featureOut = f"""/home/sspiegel/CapstoneData/Paris/training_10_classes/pickleFiles/cylinder/testing/{getTime}_Lille2_cylinder_r_{str(test["r"]).split('.')[0]}_{str(test["r"]).split('.')[1]}_grid_{str(test["grid_size"]).split('.')[0]}_{str(test["grid_size"]).split('.')[1]}_features.npz"""
# pickleOutPC = f"""/home/sspiegel/CapstoneData/Paris/Toronto_3D/pickleFiles/{getTime}_L001_r_{str(test["r"]).split('.')[0]}_{str(test["r"]).split('.')[1]}_grid_{str(test["grid_size"]).split('.')[0]}_{str(test["grid_size"]).split('.')[1]}_pointcloud.pkl"""

# outDict = { "features" : bigList, "classification" : cls}
# # outDict = {"PointCloud" : pcOut}


# with open(pickleOut, 'wb') as f:
#     pickle.dump(outDict, f)

np.savez(featureOut,array1 = pcOut, array2 = bigList, array3 = cls)

## Save stats

In [ ]:
sts = f"""/home/sspiegel/CapstoneData/Paris/training_10_classes/stats/{datetime.now(timezone.utc).strftime("%Y_%m_%d")}_cylinder_r_{str(test["r"]).split('.')[0]}_{str(test["r"]).split('.')[1]}_grid_{str(test["grid_size"]).split('.')[0]}_{str(test["grid_size"]).split('.')[1]}_features.json"""


In [ ]:
jsStats = {
    "filename" : ROOT,
    "radius" : test["r"],
    "grid_size" : test["grid_size"],
    "total_time" : "%.4f seconds" % (e - s)
}

In [ ]:
with open(sts, "w") as jsOut:
    jsOut.write(json.dumps(jsStats))
